In [1]:
%pylab inline
import scipy
import numpy as np
from pyiga import bspline, assemble, geometry, vis

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


ImportError: /home/yonnss/anaconda3/envs/pytorch/lib/python3.10/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /home/yonnss/anaconda3/envs/pytorch/lib/python3.10/site-packages/pyiga/fast_assemble_cy.cpython-310-x86_64-linux-gnu.so)

In [ ]:
# define geometry
geo = geometry.quarter_annulus()

# define discretization space
p = 3     # spline degree
kvs = 2 * (bspline.make_knots(p, 0.0, 1.0, 50),)

geo = geometry.tensor_product(geometry.BSplineFunc(kvs[0], np.linspace(0, 1,kvs[0].numdofs)), geometry.BSplineFunc(kvs[1], np.linspace(0,1,kvs[1].numdofs)))

In [ ]:
g = lambda x,y: np.exp(-(x-.8)**2 - (y-.8)**2)  # Dirichlet BC function
f = lambda x,y: 0*x # source term

# compute Dirichlet boundary conditions
bcs = assemble.compute_dirichlet_bcs(kvs, geo, [('left', g), ('top', g), ('right', g), ('bottom', g)])

# compute right-hand side from function f
rhs = assemble.inner_products(kvs, f, f_physical=True, geo=geo).ravel()

In [ ]:
# set up and solve linear system
A = assemble.stiffness(kvs, geo)
LS = assemble.RestrictedLinearSystem(A, rhs, bcs)

u = scipy.sparse.linalg.spsolve(LS.A, LS.b)
u_func = geometry.BSplineFunc(kvs, LS.complete(u))

In [ ]:
# plot coefficients and solution
figsize(12,6)

subplot(1,2,1)
pcolormesh(u_func.coeffs)
gca().set_aspect('equal')

subplot(1,2,2)
vis.plot_field(u_func, geo)
gca().set_aspect('equal')
colorbar();
import matplotlib.pyplot as plt
plt.savefig('quarter_annulus.png', dpi=300)

In [ ]:
print("Norm system: ", scipy.linalg.norm(LS.A @ u - LS.b))
n = int(np.sqrt(u_func.coeffs.size))
bd_mask = np.ones([n, n])
bd_mask[1:-1, 1:-1] = 0
bd_mask = bd_mask.flatten()
int_mask = 1 - bd_mask
int_mask = int_mask.flatten()
Pin = scipy.sparse.diags(int_mask, 0, shape=(n*n, n*n), format='csc')
Pbd = scipy.sparse.diags(bd_mask, 0, shape=(n*n, n*n), format='csc')
uc = u_func.coeffs.flatten()
res = Pin@(A@(Pin@uc)) - Pin@(-A@(Pbd@uc))
print("Norm residual: ", scipy.linalg.norm(res, ord=np.inf))

print(u_func.coeffs.shape, LS.complete(u).shape, A.shape, rhs.shape)